# **Pràctica 3: n-Grames Oberts i Classificador Ingenu de Bayes**

Aquest exercici explora els **n-Grames Oberts** com a **representació de paraules**, demostrant la seva robustesa davant del soroll i del desordre.

Definirem un **$n$-grama** com una configuració específica de $n$ lletres consecutives dins d'una paraula.

+ Per exemple, `at` i `ió` són alguns dels bigrames ($2$-grames) que formen part de la paraula `atenció`. Fent servir aquest concepte, podriem dir que la paraula `atenció` es pot representar pel conjunt de bigrames `{at, te, en, nc, ci, ió}`.

Un **$n$-grama obert** és una configuració que defineix un subconjunt ordenat —però no necessàriament contigu— de $n$ lletres dins d'una paraula.

+ Per exemple, si considerem la paraula `hello`, el conjunt de bigrames oberts és `{he, hl, ho, el, eo, ll, lo}`.
+ En moltes ocasions també és útil considerar el començament i el final d’una paraula com a casos especials, tenint en compte l’espai en blanc.
En aquest cas, per exemple, la paraula `hello` genera el següent conjunt ampliat de bigrames oberts:
`{ _h, he, hl, ho, el, eo, ll, lo, o_ }`.

> El cervell humà pot llegir frases amb les lletres internes desordenades sempre que: La primera i l’última lletra de cada paraula es mantinguin al lloc, la paraula tingui una longitud suficient, el context de la frase sigui clar. Això passa perquè, quan llegim, no analitzem cada lletra una per una, sinó que reconeixem les paraules com a formes globals i utilitzem el context semàntic per omplir els buits. El cervell fa una mena de “correcció automàtica” basant-se en les paraules que espera veure.

> Exemple: `Segns un etsdui de la Uinveristtat de Cmarbigde, el crevell pot lgegir paaulebs ambl les lleterres barajades i mab srrol sense mases dfiicultats.`

> Els n-grames oberts (open n-grams) donen una explicació molt natural de per què podem llegir/reconeixer paraules amb les lletres internes desordenades.

La definició dels $n$-grames oberts es basa en dos aspectes clau:
+ Ordenació: Les lletres han d’aparèixer en el mateix ordre que en la paraula original (per exemple, `NBK` és un 3-grama obert vàlid de la paraula `NOTEBOOK`, però `KBN` no ho és, perquè la $\text{K}$ apareix després de la $\text{B}$ i la $\text{N}$ a la paraula).
+ No contigüitat: Les lletres poden estar separades per qualsevol nombre d’altres lletres (això és el que fa que la representació sigui robusta davant de soroll, com ara errors tipogràfics o lletres que falten).

El nombre total de $n$-grames oberts per a una paraula de $L$ lletres ve donat pel coeficient binomial $\binom{L}{n}$.

Per a `NOTEBOOK` ($L=8$ i $n=3$), el total és:

$$\binom{8}{3} = \frac{8!}{3!(8-3)!} = \frac{8 \times 7 \times 6}{3 \times 2 \times 1} = \mathbf{56}$$.

In [ ]:
n-grama ordenat vol dri que, les lletres que hem passat no les farem servir, fem servir _ per representar un espai

## Part 1: Extracció de Característiques — $N$-grames oberts

### Tasca 1

Implementa una per extreure els $n$-grames oberts d’una paraula, que ens servirà com un nou tipus de representació de les paraules d'un text.

In [4]:
from itertools import combinations

def get_open_ngrams(word: str, n: int, include_boundaries: bool = True) -> set:
    """Genera un conjunt d’n-grams oberts per a una paraula donada, amb un tractament específic dels límits.

    Args:
        word (str): La paraula d’entrada.
        n (int): L’ordre de l’n-gram (per exemple, 2 per a bigrames, 3 per a trigrames).
        include_boundaries (bool): Si és True, afegeix un guió baix '_' al principi i al
                                final de la paraula per incloure els límits inicial/final.

    Returns:
        set: Un conjunt d’n-grams oberts únics.
    """
    
    open_ngrams = set()

    # 1. Generar N-grams sense boundaries (Core)
    # combinations retorna tuples, fem servir join per convertir-les a string
    if len(word) >= n:
        for ngram_tuple in combinations(word, n):
            open_ngrams.add("".join(ngram_tuple))

    # 2. Generar boundaries (si es demana)
    if include_boundaries:
        
        # Cas especial: n == 1
        # Simplement afegim el marcador de límit "_"
        if n == 1:
            open_ngrams.add("_")
        
        # Casos generals (n >= 2)
        else:
            # Regla: '_' + primer caràcter + (n-2 caràcters de la resta)
            # Això assegura que el límit només toca el primer caràcter real.
            first_char = word[0]
            rest_of_word = word[1:]
            
            # Necessitem triar (n-2) caràcters de la resta de la paraula
            # Si n=2, triem 0 caràcters (el bucle s'executa una vegada amb string buit)
            if len(rest_of_word) >= (n - 2):
                for sub_combo in combinations(rest_of_word, n - 2):
                    ngram = "_" + first_char + "".join(sub_combo)
                    open_ngrams.add(ngram)

            # Regla: (n-2 caràcters de l'inici fins al penúltim) + últim caràcter + '_'
            last_char = word[-1]
            start_of_word = word[:-1]
            
            if len(start_of_word) >= (n - 2):
                for sub_combo in combinations(start_of_word, n - 2):
                    ngram = "".join(sub_combo) + last_char + "_"
                    open_ngrams.add(ngram)

    return open_ngrams

Executa el test de la funció `get_open_ngrams` amb la paraula `hello` i comprova que funciona:

In [5]:
word_to_test = "hello"
n_gram_order_test = 2

ngrams_with_boundaries = get_open_ngrams(word_to_test, n_gram_order_test, include_boundaries=True)

print(ngrams_with_boundaries)

assert ngrams_with_boundaries == {'_h', 'el', 'eo', 'he', 'hl', 'ho', 'll', 'lo', 'o_'}

{'ho', 'he', 'hl', 'el', 'eo', 'll', 'o_', 'lo', '_h'}


## Part 2: Col·lisions entre paraules

Podem avaluar la capacitat per representar paraules comprovant la **taxa de col·lisió de paraules**, que es produeix quan dues paraules diferents tenen el mateix conjunt de característiques.

Per fer-ho farem servir el fitxer `dataset.csv`, que conté frases en diferents idiomes:

In [6]:
import pandas as pd
from collections import Counter
import os

dataset_file_path = 'dataset.csv'

try:
    df = pd.read_csv(dataset_file_path)

    # Extracció de textes i etiquetes
    texts = df['Text'].tolist()
    labels = df['language'].tolist()

    print("Fitxer carregat.")
    print(f"Nombre de frases: {len(texts)}")
    print(f"Exemple de frase: {texts[0]}")
    print(f"Exemple d'etiqueta: {labels[0]}")
    print("\nDistribució de Llengues:")
    print(Counter(labels))

except FileNotFoundError:
    print(f"Error: Fitxer '{dataset_file_path}' no trobat.")
    print("Assegura't que dataset.csv està accessible.")
except Exception as e:
    print(f"Error al carregar el fitxer: {e}")

Fitxer carregat.
Nombre de frases: 22000
Exemple de frase: klement gottwaldi surnukeha palsameeriti ning paigutati mausoleumi surnukeha oli aga liiga hilja ja oskamatult palsameeritud ning hakkas ilmutama lagunemise tundemärke  aastal viidi ta surnukeha mausoleumist ära ja kremeeriti zlíni linn kandis aastatel – nime gottwaldov ukrainas harkivi oblastis kandis zmiivi linn aastatel – nime gotvald
Exemple d'etiqueta: Estonian

Distribució de Llengues:
Counter({'Estonian': 1000, 'Swedish': 1000, 'Thai': 1000, 'Tamil': 1000, 'Dutch': 1000, 'Japanese': 1000, 'Turkish': 1000, 'Latin': 1000, 'Urdu': 1000, 'Indonesian': 1000, 'Portugese': 1000, 'French': 1000, 'Chinese': 1000, 'Korean': 1000, 'Hindi': 1000, 'Spanish': 1000, 'Pushto': 1000, 'Persian': 1000, 'Romanian': 1000, 'Russian': 1000, 'English': 1000, 'Arabic': 1000})


In [7]:
# AMb el dataset trobarem un index perr dir quina llengua és, en el segon sheet creo, i una frase en aquell idioma
# Fer primers situacions de n grames de 


### Tasca 2

Escriu un codi en Python que:

1. Extreu totes les paraules úniques de `texts` i posa-les a un conjunt que es dirà `unique_words`
2. Genera, per a cada paraula, els bigrames oberts ($n$=2) i conta quants conjunts de bigrames únics hi ha al dataset.
3. Detecta si hi ha col·lisions al conjunt de dades, és a dir, casos en què diferents paraules tenen exactament el mateix conjunt d’$n$-grames.
4. Imprimeix:
    + El total de paraules **úniques** extretes.
    + El total de conjunts **únics** d’n-grames generats.
    + El nombre de paraules úniques implicades en col·lisions.
    + 5 exemples de parellles de paraules en col·lisió.

In [8]:

import re
from collections import defaultdict

# 1. Extreu totes les paraules úniques de 'texts'
unique_words = set()

for text in texts:
    if isinstance(text, str): # Verificació de seguretat per si hi ha NaNs
        # Utilitzem regex per trobar paraules (\w+), convertim a minúscules
        # per evitar que "Hola" i "hola" siguin diferents.
        words = re.findall(r'\b\w+\b', text.lower())
        unique_words.update(words)

# 2. Genera els bigrames oberts (n=2) amb límits per a cada paraula única
# Utilitzem un diccionari on:
# Clau: Una representació immutable del conjunt d'n-grams (frozenset o tuple ordenada)
# Valor: Llista de paraules que generen aquest conjunt
ngram_signature_map = defaultdict(list)

for word in unique_words:
    # Generem el set
    ngrams_set = get_open_ngrams(word, n=2, include_boundaries=True)
    
    # Convertim el set a una tupla ordenada per poder usar-la com a clau de diccionari (hashable)
    ngrams_signature = tuple(sorted(list(ngrams_set)))
    
    # Guardem la paraula sota aquesta signatura
    ngram_signature_map[ngrams_signature].append(word)

# 3. Identifica i compta les col·lisions
# Una col·lisió existeix si la llista de paraules per a una clau té longitud > 1
collision_groups = []
words_in_collision_count = 0

for signature, words_list in ngram_signature_map.items():
    if len(words_list) > 1:
        collision_groups.append(words_list)
        words_in_collision_count += len(words_list)

# 4. Compta i imprimeix resultats
total_unique_words = len(unique_words)
total_unique_ngram_sets = len(ngram_signature_map)


print(f"Total paraules úniques extretes: {total_unique_words}")
print(f"Total conjunts únics d'n-grames: {total_unique_ngram_sets}")
print(f"Nombre de paraules implicades en col·lisions: {words_in_collision_count}")

# Càlcul del percentatge de col·lisió
if total_unique_words > 0:
    percentatge = (words_in_collision_count / total_unique_words) * 100
    print(f"Percentatge de col·lisió: {percentatge:.2f}%")

print("\n5 exemples de grups de paraules en col·lisió:")
# Ordenem per longitud del grup per veure col·lisions interessants, o agafem els primers 5
# Aquí mostrem els primers 5 trobats:
for i, group in enumerate(collision_groups[:5]):
    print(f"  {i+1}. {group} -> Comparteixen els mateixos bigrames")





Total paraules úniques extretes: 278630
Total conjunts únics d'n-grames: 277959
Nombre de paraules implicades en col·lisions: 1335
Percentatge de col·lisió: 0.48%

5 exemples de grups de paraules en col·lisió:
  1. ['posibilidades', 'possibilidades'] -> Comparteixen els mateixos bigrames
  2. ['assinaram', 'assassinaram'] -> Comparteixen els mateixos bigrames
  3. ['للتحليل', 'لتحليل'] -> Comparteixen els mateixos bigrames
  4. ['concursuum', 'concursum'] -> Comparteixen els mateixos bigrames
  5. ['ссср', 'сср'] -> Comparteixen els mateixos bigrames


In [ ]:
#FUNCIÓ DEL CODI SUPERIOR

import re
from collections import defaultdict

def analitza_collisions_ngrams(texts, n=2, include_boundaries=True, max_examples=5):
    """
    Analitza col·lisions de conjunts d'open n-grams per paraula.

    Retorna un diccionari amb:
      - total_unique_words
      - total_unique_ngram_sets
      - words_in_collision_count
      - percentatge_col_lisio
      - collision_groups (llista de llistes de paraules)
    """
    # 1. Extreu totes les paraules úniques de 'texts'
    unique_words = set()

    for text in texts:
        if isinstance(text, str):
            words = re.findall(r'\b\w+\b', text.lower())
            unique_words.update(words)

    # 2. Genera els n-grams oberts per a cada paraula única
    ngram_signature_map = defaultdict(list)

    for word in unique_words:
        ngrams_set = get_open_ngrams(word, n=n, include_boundaries=include_boundaries)
        ngrams_signature = tuple(sorted(list(ngrams_set)))
        ngram_signature_map[ngrams_signature].append(word)

    # 3. Identifica col·lisions
    collision_groups = []
    words_in_collision_count = 0

    for signature, words_list in ngram_signature_map.items():
        if len(words_list) > 1:
            collision_groups.append(words_list)
            words_in_collision_count += len(words_list)

    # 4. Càlcul d’estadístiques
    total_unique_words = len(unique_words)
    total_unique_ngram_sets = len(ngram_signature_map)

    percentatge_col_lisio = 0.0
    if total_unique_words > 0:
        percentatge_col_lisio = (words_in_collision_count / total_unique_words) * 100

    # Opcional: imprimir alguns resultats
    print(f"Total paraules úniques extretes: {total_unique_words}")
    print(f"Total conjunts únics d'n-grames: {total_unique_ngram_sets}")
    print(f"Nombre de paraules implicades en col·lisions: {words_in_collision_count}")
    print(f"Percentatge de col·lisió: {percentatge_col_lisio:.2f}%")

    print(f"\n{max_examples} exemples de grups de paraules en col·lisió:")
    for i, group in enumerate(collision_groups[:max_examples]):
        print(f"  {i+1}. {group} -> Comparteixen els mateixos n-grames")

    # Retornem les dades per si les vols reutilitzar
    return {
        "total_unique_words": total_unique_words,
        "total_unique_ngram_sets": total_unique_ngram_sets,
        "words_in_collision_count": words_in_collision_count,
        "percentatge_col_lisio": percentatge_col_lisio,
        "collision_groups": collision_groups,
    }


Passos i pasos colisiona pq té igual length i igual ngrames

## Part 2: Classificador Ingenu de Bayes

L'objectiu és fer un classificador que sigui capaç de predir la llengua d'una frase a partir de representar la frase com el conjunt de bigrames de les seves paraules.

Per fer-ho començarem dividint el nostre dataset en una part de *training* i una de *test*.

In [2]:
#Empiesa la wea seria


In [16]:
### 

from sklearn.model_selection import train_test_split

# Les diferentcies entre x i y:
"""
texts és frase, label idioma, 0.2 per a test

Parametre  shufle pot ser interessant per barrejar les dades

X train és sense respostes, sense els idiomes el X test són les seves respostes (el 0.8)

Y train és també sense respostes, però del 0.2
"""
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

print(f"Training texts length: {len(X_train)}")
print(f"Test texts length: {len(X_test)}")
print(f"Training labels length: {len(y_train)}")
print(f"Test labels length: {len(y_test)}")

Training texts length: 17600
Test texts length: 4400
Training labels length: 17600
Test labels length: 4400


### Tasca 3: Preparació de les dades
1. Extreu els bigrames oberts de cada un dels textos del conjunt d'entrenament (els bigrames oberts d'un text és el resultat de la unió dels conjunts de bigrames de totes les seves paraules).
2. Un cop calculats, representa cada text d'entrenament amb una llista (`X_train_ngrams[:][:]`) de bigrames enlloc d'un conjunt. El primer índex és l'índex del text i el segon la llista dels seus bigrames.
3. Mostra la llista que correspon al primer text d'entrenament.

Indicació: Per extreure totes les paraules d'un text pots fer servir aquest codi:

```python
text_lower = text.lower()
words_in_text = re.findall(r'\b\w+\b', text_lower)
```

X trains és llista de llistes


Hem de construir en index 0 els bigrames oberts

X_train[11] ha de sortir tonalli es un nombre personal...

x_train_ngrams[], tindrà TOTS els bigrames oberts unics de la frase 11



In [17]:
# Extreu els n-grames oberts dels textos del conjunt de training a la variable X_train_ngrams[:][:]
import re

# Xtrain és una llista de textos (str)
# Retorna X_train_ngrams: llista de llistes de bigrames per cada text
def get_all_ngrams_from_text(Xtrain, n: int, include_boundaries: bool = True):
    X_train_ngrams = []  # llista de textos, cada element serà la llista de bigrames únics d'aquell text

    for text in Xtrain:
        text_lower = text.lower()
        words_in_text = re.findall(r'\b\w+\b', text_lower)

        # conjunt per fer la unió de bigrames de totes les paraules del text
        ngrams_set = set()

        for word in words_in_text:
            word_ngrams = get_open_ngrams(word, n, include_boundaries)
            ngrams_set.update(word_ngrams)

        # aquest text queda representat com la llista dels seus bigrames únics
        X_train_ngrams.append(list(ngrams_set))

    return X_train_ngrams


Extreu els n-grames oberts dels textos del conjunt de test (`X_test_ngrams`[:][:]).


In [ ]:
# Extreu els n-grames oberts dels textos del conjunt de test a la variable X_test_ngrams[:][:]
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)
X_train_ngrams=get_all_ngrams_from_text(X_train, n=2, include_boundaries=True)

Contar quant importants són per a cada idioma cada bigrama per posar-hi prob

tf alt però id baix al i el apareix al dues frases un cop.

sklearn té tf i idf vectorizer


amb el lambda de tf indiquem que pasem raw content i no una folder o un path


Les compressed sparse permet no haver de guardar tant zeros, sinó guarda info dels elements que són 0


Normalemnt les sparse es treballen amb scipy



Més endavant (T4)

c són idiomes, d és document


En aquest data set totes tenen == prob, però cal calcular



w és bigrama

donar bigrama w_i donam prob de que sigui c

Pensa en lbbar quantes vegades tingui, el meu idioma, aquest bigrama


Quan hi ha moltes multiplicacions, fer ús de logaritmes per estalviar espai



Quan diu paraula pensa que és un bigrama, error tipo

Per comprovar els cosos scipy té la llibreria MultinomialNB creo

Com scipy es open source podem agafar idees, però no podem copiar 
diu k no funcionara ns loko


Detecta llengues en 95% accuracy aprox

És més facil fer Multinomial com a classe

Fer una CustomMultinomialNB


class i feature són prob cndicionada

classes idiomes

nfeatures bigrames





Mètode fit


fit és per entrenar un model

Hem d'aplicar formula bayes naive
Però en aquest context si no parem de multiplicar hi haura tri 0, llavors agafarem logaritmes pq mul == suma, potencia == mul

argmax és de totes les probabilitatss agafar la més gran, que serà l'idioma que ell haurà suposat del text

FIT (XTRAIN Y TRAIN)

TRAIN (XTEST Y TEST)

funcions de multiNB




Si no tenim en compte el tipus de dades tindrem overflow (allocate) Necessitarem float32 (perdem precissió  a canvi de q funcioni)

fit anirà iiterant per a cada idioma


EL +1 DE P(w_i | c) és un smoothing de Laplace per evitar que tots els 0 me tri jodan la mandanga, pots provar diferents per veure com canvia




pensa que el MultiNBfit ha de retornar self # si es treballa en classes no cal retornar res el retirn self és simplement per indicar que no retornem res més que el model entrenat
Més per clean code que altre wea



Al final, el log, hem de mirar on guardar la probabilitat i és lliure

Ell fa una llista xtest (llista de llistes [i,:]
per tal que a index 0 tinc 1r idioma i així


predict log ns que de argmax


Ha de sortir 0.9545 el pred_custom -> Diu que si surt 90 o 80 tampoc passa res
Però ns

Treballar amb
float32, treballar @, 

El següent objectiu és aplicar un **classificador ingenu de Bayes** per detectar les llegües dels textos del conjunt de test.

Per fer-ho hem de convertir els $n$-grames oberts extrets en **representacions numèriques** i ho farem amb el mètode TF-IDF, preparant-los per al classificador.

> El **TF-IDF** (de Term Frequency – Inverse Document Frequency) és una tècnica molt utilitzada en processament del llenguatge natural per representar textos de manera numèrica i mesurar la importància de cada element del text (en el nostre cas bigrames) dins d’un conjunt de documents.

TF-IDF combina dues idees simples:

+ TF — Term Frequency (freqüència del terme):  Mesura quantes vegades apareix un element o terme dins d’un document.

$$TF(t, d) = \frac{\text{nombre de vegades que el terme } t \text{ apareix a } d}{\text{nombre total de paraules al document } d}$$

+ IDF — Inverse Document Frequency (freqüència inversa del document): Mesura com d’especial és un element dins del conjunt total de documents.

$$ IDF(t) = \log\left(\frac{N}{1 + n_t}\right) $$

On:
+ $N$ = nombre total de documents
+ $n_t$ = nombre de documents on apareix l'element $t$

👉 Si una paraula (o bigrama) apareix a gairebé tots els documents (com `el`, `una`, `de`), el seu IDF és baix.

👉 Si només apareix en pocs, el seu IDF és alt — i, per tant, és més discriminativa.

El **TF-IDF** és:

$$TF\text{-}IDF(t, d) = TF(t, d) \times IDF(t)$$

Així, el elements:
+ freqüents dins d’un text (alt TF)
+ però poc freqüents en la resta del corpus (alt IDF)

reben més pes en la representació numèrica.

Suposa dos textos:
1. “El gat dorm al sofà.”
2. “El gos juga al parc.”

Les paraules `el` i `al` apareixen a tots dos → TF alt però IDF baix.
Les paraules `gat`, `gos`, `sofà` o `parc` apareixen només a un → TF moderat però IDF alt → més importants per diferenciar els textos.



In [13]:
# TF-IDF ja està inclòs a `scikit-learn`, una de les llibreries més
# populars per a machine learning i el podem fer servir així en el
# nostre cas:

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(analyzer=lambda x: x)

vectorizer.fit(X_train_ngrams)

X_train_tfidf = vectorizer.transform(X_train_ngrams)
X_test_tfidf = vectorizer.transform(X_test_ngrams)

print("TF-IDF vectorization complete.")
print(f"Shape of X_train_tfidf: {X_train_tfidf.shape}")
print(f"Shape of X_test_tfidf: {X_test_tfidf.shape} \n")

NameError: name 'X_train_ngrams' is not defined

Les matrius que ha generat la cel·la anterior estan guardats amb una estructura de python que es diu "Compressed Sparse Row sparse matrix".

Pensa que tens una taula enorme de nombres, però la majoria són zeros. Guardar tots aquests zeros és una pèrdua de memòria i de temps.

Per això en Python (amb scipy) sovint es fan servir matrius disperses (sparse matrices), i una de les més habituals és el format Compressed Sparse Row (CSR).

En lloc de guardar tots els elements de la matriu, una CSR només guarda:
+ Els valors no zero
+ La columna de cada valor no zero
+ On comença i acaba cada fila dins d’aquestes llistes

Però tu, com a usuari, no cal que et preocupis gaire de com ho fa per dins:
la tractes gairebé com si fos una matriu de NumPy, però amb algunes diferències.

### Tasca 4: Implementació d'un classificador ingenu de Bayes

Has d'entrenar un classificador Naive Bayes amb les característiques TF-IDF del conjunt d'entrenament i les seves etiquetes de llengua corresponents.

El model calcula la probabilitat que un text $d$ pertanyi a una llengua $c$:

$$P(c \mid d) \propto P(c) \prod_{i=1}^{V} P(w_i \mid c)^{\, f_i}$$

On:
+ $P(c)$: probabilitat prèvia de la classe (per exemple, % de textos d'una llengua; en el nostre cas totes les llengües tenen la mateixa probabilitat).
+ $w_i$: bigrama i-èssim del vocabulari.
+ $f_i$: nombre de vegades que el bigrama $w_i$ apareix al text $d$.
+ $P(w_i \mid c)$: probabilitat que el bigrama $w_i$ aparegui en textos de la classe $c$.
+ $V$: mida del vocabulari.

El classificador escull la classe amb probabilitat més alta.

#### Com es calcula $P(w_i \mid c)$?

Normalment es calcula amb un model multinomial amb Laplace *smoothing* (per evitar zeros):

$$P(w_i \mid c) =
\frac{N_{i,c} + 1}{\sum_{j=1}^{V} N_{j,c} + V}$$

On:
+ $N_{i,c}$: nombre total de vegades que la paraula $w_i$ apareix en tots els documents de la classe c.
+ $V$: nombre total de paraules diferents del vocabulari.

Aquesta fórmula ve directament de la distribució multinomial, perquè compta freqüències de paraules dins d’una “bossa” pròpia de cada classe.

Fes una implementació teva del model multinomial, és a dir, de les funcions que creen el model i apliquen el model al test:

 `MultinomialNBfit(X_train_tfidf, y_train))`

 i

 `MultinomialNBpredict(model, X_test_tfidf, y_test))`

Imprimeix quina `accuracy` obtens. Si tot funciona correctament, hauries d'aconseguir una *accuracy* per sobre el 90%.

In [ ]:
def MultinomialNBfit(X, y):
        """
        Entrena el classificador Naive Bayes.

        Args:
            X (sparse matrix): Matriu de característiques (e.g., TF-IDF) d'entrenament.
            y (array-like): Etiquetes de classe per a cada mostra.
        """
        return model

def MultinomialNBpredict(model,X, y):
        """
        Aplica el classificador Naive Bayes.

        Args:
            X (sparse matrix): Matriu de característiques (e.g., TF-IDF) de test.
            y (array-like): Etiquetes de classe per a cada mostra del test.
        """
        return accuracy_score

print(accuracy_score)

In [ ]:
Aniran a buscar eso a funcions implementades aqui o altres no implementades a skilearn

Jugar amb els parametres del argmax i el smoothing i provar de petar algun idioma

Si tinc model a preedir sempre angles que creus que pot ser que ho ocasioni (que és simplificat i facil)

